# Preparations

Import various libraries: Mainly Tensorflow & Keras

> **TODO: Remove unnecessary imports**

In [1]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential,Input,Model
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,Concatenate,GlobalAveragePooling2D,Lambda,ZeroPadding2D
import matplotlib.pylab as plot
import pandas as pd
%matplotlib inline
import matplotlib.pylab as plt
import seaborn as sns

Using TensorFlow backend.


Setting some general values.

In [2]:
IMG_HEIGHT = 200 # original is 300
IMG_WIDTH = 200

BATCH_SIZE_TRAIN = 10
BATCH_SIZE_VAL = 10
EPOCHS = 20

# Load the Images

First we load the classified images.

In [3]:
dataset_dir = './dataset/'

img_gen = ImageDataGenerator(
    rescale = 1./255, # Rescale [0, 255] to [0, 1] because NN prefer smaller input
    horizontal_flip = True, # Randomly flip inputs horizontally
    vertical_flip = True, # Randomly flips inputs vertically
    height_shift_range = .2,
    validation_split = 0.2 # mark  20% as belonging to the validation set
)

train_gen = img_gen.flow_from_directory(
    dataset_dir,
    batch_size=BATCH_SIZE_TRAIN,
    shuffle = True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode = 'categorical',
    subset = 'training'
)

val_gen = img_gen.flow_from_directory(
    dataset_dir,
    batch_size=BATCH_SIZE_VAL,
    shuffle = True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode = 'categorical',
    subset = 'validation'
)

# print(train_gen[0])


Found 1751 images belonging to 3 classes.
Found 437 images belonging to 3 classes.


We have 2188 images in total.
They have now been split up in training and validation sets (that are strictly not overlapping)

# Create the model

This is the part that has to be optimized the most! (and isn't at all yet)

In [4]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(3, activation='sigmoid')
])

# Compile the model

In [5]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 200, 200, 16)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 100, 100, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 32)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 25, 25, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 25, 25, 64)       

In [ ]:
history = model.fit_generator(
    train_gen,
    steps_per_epoch=BATCH_SIZE_TRAIN,
    epochs=EPOCHS,
    validation_data=val_gen,
    validation_steps=BATCH_SIZE_VAL
)

Epoch 1/20
10/10 [==============================] - 7s 680ms/step - loss: 0.6725 - accuracy: 0.6400 - val_loss: 0.8365 - val_accuracy: 0.6667
Epoch 2/20
10/10 [==============================] - 5s 534ms/step - loss: 0.6563 - accuracy: 0.6667 - val_loss: 0.6149 - val_accuracy: 0.6667
Epoch 3/20
10/10 [==============================] - 6s 600ms/step - loss: 0.6326 - accuracy: 0.6667 - val_loss: 0.5830 - val_accuracy: 0.6667
Epoch 4/20
10/10 [==============================] - 5s 535ms/step - loss: 0.6002 - accuracy: 0.6667 - val_loss: 0.6472 - val_accuracy: 0.6067
Epoch 5/20
10/10 [==============================] - 5s 548ms/step - loss: 0.6077 - accuracy: 0.6667 - val_loss: 0.5959 - val_accuracy: 0.6976
Epoch 6/20
10/10 [==============================] - 6s 557ms/step - loss: 0.5815 - accuracy: 0.6567 - val_loss: 0.6070 - val_accuracy: 0.7400
Epoch 7/20
10/10 [==============================] - 5s 516ms/step - loss: 0.4562 - accuracy: 0.8168 - val_loss: 0.4429 - val_accuracy: 0.8100
Epoch 

Evaluating the result

In [ ]:
scores = model.evaluate_generator(val_gen, steps=BATCH_SIZE_VAL)
print("Model Test Loss:" , scores[0])
print("Model Test Accuracy:", scores[1])

# Saving the model

We're using the SavedModel format as that also saves all of the weights 

In [ ]:
model_name = "akt-model"

model.save(model_name + ".h5")
# can be loaded again with
# from keras.models import load_model
# model = load_model(model_name + ".h5")

# Evaluating the model

The following few steps will output different information about the model

In [ ]:
plot.plot(history.history['loss'])

In [ ]:
plot.plot(history.history['accuracy'])

In [ ]:
predictions = model.predict_generator(val_gen)

confusion_predicts = list(map(lambda p: np.argmax(p), predictions))

con_mat = tf.math.confusion_matrix(val_gen.labels, confusion_predicts, 3).numpy()
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)
con_mat_df = pd.DataFrame(con_mat_norm, index = [0, 1, 2], columns = [0, 1, 2])
plt.figure(figsize=(5, 5))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.YlGn)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()